In [1]:
library(rlang)
library(purrr)


Attaching package: ‘purrr’

The following objects are masked from ‘package:rlang’:

    %@%, %||%, as_function, flatten, flatten_chr, flatten_dbl,
    flatten_int, flatten_lgl, invoke, list_along, modify, prepend,
    rep_along, splice



### Motivation

Imagine you’re creating a lot of strings by joining together words:

In [4]:
cement <- function(...) {
  args <- ensyms(...)
  paste(purrr::map(args, as_string), collapse = " ")
}

cement(Good, morning, Hadley)
#> [1] "Good morning Hadley"
cement(Good, afternoon, Alice)
#> [1] "Good afternoon Alice"

[1] "Good morning Hadley"

[1] "Good afternoon Alice"

In [5]:
name <- "Hadley"
time <- "morning"

paste("Good", time, name)
#> [1] "Good morning Hadley"

[1] "Good morning Hadley"

this doesn’t work with cement() because every input is automatically quoted:

In [6]:
cement(Good, time, name)
#> [1] "Good time name"

[1] "Good time name"

Quasiquotation gives us a standard tool to do so: `!!`, called “unquote”, and pronounced bang-bang. `!!` tells a quoting function to drop the implicit quotes:

In [8]:
cement(Good, !!time, !!name)
#> [1] "Good morning Hadley"

[1] "Good morning Hadley"

If you’re ever unsure about whether an argument is quoted or evaluated, try executing the code outside of the function. If it doesn’t work (or does something profoundly different), then that argument is quoted. For example, you can use this technique to determine that the first argument to `library()` is quoted:

In [11]:
# works
library(MASS)

# fails
MASS
#> Error in eval(expr, envir, enclos):
#>   object 'MASS' not found

ERROR: Error in eval(expr, envir, enclos): object 'MASS' not found


Talking about whether an argument is quoted or evaluated is a more precise way of stating whether or not a function uses non-standard evaluation (NSE).

`expr()` is great for interactive exploration, because it captures what you, the developer, typed. It’s not so useful inside a function:

In [13]:
f1 <- function(x) expr(x)
f1(a + b + c)
#> x

x

We need another function to solve this problem: `enexpr()`. This captures what the caller supplied to the function by looking at the internal promise object that powers lazy evaluation

In [15]:
f2 <- function(x) enexpr(x)
f2(a + b + c)
#> a + b + c

a + b + c

To capture multiple arguments (e.g. all arguments in `...`), use `enexprs()`.

In [16]:
f <- function(...) enexprs(...)
f(x = 1, y = 10 * z)
#> $x
#> [1] 1
#> 
#> $y
#> 10 * z

$x
[1] 1

$y
10 * z


Finally, `exprs()` is useful interactively to make a list of expressions:

In [22]:
exprs(x = x ^ 2, y = y ^ 3, z = z ^ 4)

$x
x^2

$y
y^3

$z
z^4


In short, use `enexpr()` and `enexprs()` to capture the expressions supplied as arguments by the user. Use `expr()` and `exprs()` to capture expressions that you supply.

### Capturing symbols

Sometimes you only want to allow the user to specify a variable name, not an arbtirary expression. In this case, you can use `ensym()` or `ensyms()`. These are variants of `enexpr()` and `enexprs()` that check the captured expression is either symbol or a string (which is converted to symbol64).

In [30]:
f <- function(...) ensyms(...)
f(x)
#> [[1]]
#> x
f("x")
#> [[1]]
#> x

ERROR while rich displaying an object: Error in FUN(X[[i]], ...): attempt to use zero-length variable name

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_html.list(obj)
9. repr_list_generic(obj, "html", "\t<li>%s</li>\n", "\t<dt>$%s</dt>\n\t\t<dd>%s</dd>\n", 
 .     "<strong>$%s</strong> = %s", "<ol>\n%s</ol>\n", "<dl>\n%s</dl>\n", 
 .     numeric_item = "\t<dt>[[%s]]</dt>\n\t\t<dd>%s</dd>\n", escape_fun = html_escape)


[[1]]
x


ERROR while rich displaying an object: Error in FUN(X[[i]], ...): attempt to use zero-length variable name

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_html.list(obj)
9. repr_list_generic(obj, "html", "\t<li>%s</li>\n", "\t<dt>$%s</dt>\n\t\t<dd>%s</dd>\n", 
 .     "<strong>$%s</strong> = %s", "<ol>\n%s</ol>\n", "<dl>\n%s</dl>\n", 
 .     numeric_item = "\t<dt>[[%s]]</dt>\n\t\t<dd>%s</dd>\n", escape_fun = html_escape)


[[1]]
x


**rlang:**


  | Developer | User
-- | -- | --
One | expr() | enexpr()
Many | exprs() | enexprs()


**base R:**

  | Developer | User
-- | -- | --
One | quote() | substitute()
Many | alist() | eval(substitute(alist()))

 ### Unquoting one argument

Use `!!` to unquote a single argument in a function call. `!!` takes a single expression, evaluates it, and inlines the result in to the AST.

In [33]:
x <- expr(a + b + c)
expr(f(!!x, y))
#> f(a + b + c, y)

f(a + b + c, y)

As well as call objects, `!!` also works with symbols and constants:

In [34]:
a <- sym("y")
b <- 1
expr(f(!!a, !!b))
#> f(y, 1)

f(y, 1)

Note that the right-hand side of `!!` can be a function call. `!!` will evaluate the call and insert the results in the AST:

In [35]:
mean_rm <- function(var) {
  var <- ensym(var)
  expr(mean(!!var, na.rm = TRUE))
}
expr(!!mean_rm(x) + !!mean_rm(y))
#> mean(x, na.rm = TRUE) + mean(y, na.rm = TRUE)

mean(x, na.rm = TRUE) + mean(y, na.rm = TRUE)

Note that `!!` preserves operator precedence because it works with expressions.

In [37]:
x1 <- expr(x + 1)
x2 <- expr(x + 2)

expr(!!x1 / !!x2)
#> (x + 1)/(x + 2)

(x + 1)/(x + 2)

### Unquoting a function

`!!` is most commonly used to replace the arguments to a function, but you can also use it to replace the function itself. The only challenge here is operator precedence: `expr(!!f(x, y))` unquotes the result of `f(x, y)`, so you need an extra pair of parentheses.

In [39]:
f <- expr(foo)
expr((!!f)(x, y, z))
#> foo(x, y, z)

foo(x, y, z)

This also works when `f` is itself a call:

In [40]:
f <- expr(pkg::foo)
expr((!!f)(x, y, z))
#> pkg::foo(x, y, z)

pkg::foo(x, y, z)

Because of the large number of parentheses involved, it can be more clear to use `rlang::call2()`:

In [41]:
f <- expr(pkg::foo)
call2(f, expr(x), expr(y))
#> pkg::foo(x, y)

pkg::foo(x, y)

### Unquoting a missing argument

Very occasionally it is useful to unquote a missing argument, but the naive approach doesn’t work:

In [42]:
arg <- missing_arg()
expr(foo(!!arg, !!arg))
#> Error in enexpr(expr):
#>   argument "arg" is missing, with no default

ERROR: Error in enexpr(expr): argument "arg" is missing, with no default


You can work around this with the `maybe_missing()` helper:

In [43]:
expr(foo(!!maybe_missing(arg), !!maybe_missing(arg)))
#> foo(, )

foo(, )

### Unquoting in special forms

In [44]:
expr(df$!!x)
#> Error: unexpected '!' in "expr(df$!"

ERROR: Error in parse(text = x, srcfile = src): <text>:1:9: unexpected '!'
1: expr(df$!
            ^


Here you need to use the prefix form:

In [48]:
x <- expr(y)
expr(`$`(df, !!x))
#> df$y

df$y

### Unquoting many arguments

`!!` is a one-to-one replacement. `!!!` (called “unquote-splice”, and pronounced bang-bang-bang) is a one-to-many replacement. It takes a list of expressions and inserts them at the location of the `!!!`:

In [49]:
xs <- exprs(1, a, -b)
expr(f(!!!xs, y))
#> f(1, a, -b, y)

f(1, a, -b, y)

`!!!` can be used in any rlang function that takes `...` regardless of whether or not `...` is quoted or evaluated.

note that this can be useful in `call2()`.

In [50]:
call2("f", !!!xs, expr(y))
#> f(1, a, -b, y)

f(1, a, -b, y)